In [1]:
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import csv
import uuid
import os
import shutil
import random
%matplotlib inline

In [3]:
trainFile=open("train.csv", "w")
testFile=open("test.csv", "w")
trainWriter = csv.writer(trainFile)
testWriter = csv.writer(testFile)
trainWriter.writerow(['ImageId','EncodedPixels'])
testWriter.writerow(['ImageId','EncodedPixels'])
def resize(left,right):
    full=200-(right-left)
    if full==0:
        return left,right
    half=full//2+1
    if(left<half):
        return 0,right+full-left
    if(768-right<half):
        return left-(full-(768-right)),768
    return left-half,right+full-half
def saveImg(path,imgPixel):
    mpimg.imsave(path, imgPixel)
def clearDir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.mkdir(path)
clearDir('trainImg')
clearDir('testImg')
with open('raw3000.csv') as trainCSVFile:
    trainRawData = csv.reader(trainCSVFile)
    next(trainRawData)
    pixelMap={}
    booleanMap={}
    for row in trainRawData:
        imgPixels = np.array(mpimg.imread('rawImg/'+row[0]))
        imgId=str(uuid.uuid1())
        if row[1]=='':
            x=random.randint(0,568)
            y=random.randint(0,568)
            if random.random()<0.8:
                saveImg('trainImg/'+imgId+'.jpg', imgPixels[x:x+200,y:y+200])
                trainWriter.writerow([imgId+'.jpg','0'])
            else:
                saveImg('testImg/'+imgId+'.jpg', imgPixels[x:x+200,y:y+200])
                testWriter.writerow([imgId+'.jpg','0'])
            booleanMap[imgId]=np.array([0.,1.])
        else:
            runLengthCode=row[1].split(' ')
            pixelIndexs=[]
            left=767
            right=0
            top=int(runLengthCode[0])//768
            down=int(runLengthCode[-2])//768
            if down-top>200:
                continue
            for i in range(0,len(runLengthCode),2):
                startPixel=int(runLengthCode[i])
                runLength=int(runLengthCode[i+1])
                x=startPixel%768
                y=startPixel//768
                if x<left:
                    left=x
                if x+runLength-1>right:
                    right=x+runLength-1
                for j in range(runLength):
                    pixelIndexs.append([x+j,y])
            if right-left>200:
                continue
            left,right=resize(left,right)
            top,down=resize(top,down)
            for pixelIndex in pixelIndexs:
                pixelIndex[0]-=left
                pixelIndex[1]-=top
            if random.random()<0.8:
                saveImg('trainImg/'+imgId+'.jpg', imgPixels[left:right,top:down])
                trainWriter.writerow([imgId+'.jpg','1'])
            else:
                saveImg('testImg/'+imgId+'.jpg', imgPixels[left:right,top:down])
                testWriter.writerow([imgId+'.jpg','1'])
            booleanMap[imgId]=np.array([1.,0.])
            pixelMap[imgId]=np.array(pixelIndexs)
trainFile.close()
testFile.close()

In [5]:
def saveImg(path,imgPixel):
    mpimg.imsave(path, imgPixel)
def clearDir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.mkdir(path)
trainFile=open("trainOrigin.csv", "w")
testFile=open("testOrigin.csv", "w")
trainWriter = csv.writer(trainFile)
testWriter = csv.writer(testFile)
trainWriter.writerow(['ImageId','ship_or_not'])
testWriter.writerow(['ImageId','ship_or_not'])
clearDir('trainOriginImg')
clearDir('testOriginImg')
with open('raw1000.csv') as trainCSVFile:
    trainRawData = csv.reader(trainCSVFile)
    next(trainRawData)
    temp=''
    for row in trainRawData:
        imgPixels = np.array(mpimg.imread('rawImg/'+row[0]))
        if row[1]=='':
            if random.random()<0.8:
                saveImg('trainOriginImg/'+row[0], imgPixels)
                trainWriter.writerow([row[0],'0'])
            else:
                saveImg('testOriginImg/'+row[0], imgPixels)
                testWriter.writerow([row[0],'0'])
        else:
            if temp==row[0]:
                continue
            temp=row[0]
            if random.random()<0.8:
                saveImg('trainOriginImg/'+row[0], imgPixels)
                trainWriter.writerow([row[0],'1'])
            else:
                saveImg('testOriginImg/'+row[0], imgPixels)
                testWriter.writerow([row[0],'1'])
trainFile.close()
testFile.close()

In [4]:
# ax = plt.figure().add_axes([0, 0, 1, 1])
# imgId=list(pixelMap.keys())[0]
# imgPixels=mpimg.imread('trainImg/'+imgId+'.jpg')
# maskPixels =  [[[255,255,255] for col in range(200)] for row in range(200)]
# for pixelIndex in pixelMap[imgId]:
#     maskPixels[pixelIndex[0]][pixelIndex[1]]=[255,255,0]
# ax.imshow(imgPixels)
# ax.imshow(maskPixels,alpha=0.4)

In [5]:
# def bounds(step):
#     result=[]
#     for i in range(0, 768-200 ,step):
#         result.append(i)
#     result.append(768-200)
#     return result
# imgPixels = np.array(mpimg.imread('testImg/'+row[0]))
# for x in bounds(100):
#     for y in bounds(100):
#         print(imgPixels[x:x+200,y:y+200])

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 200,200,3])/255.
ys = tf.placeholder(tf.float32, [None, 2])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 200, 200, 3])

## conv1 layer ##
W_conv1 = weight_variable([5,5, 3,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## conv2 layer ##
W_conv2 = weight_variable([5,5, 32, 64]) # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 14x14x64
h_pool2 = max_pool_2x2(h_conv2)                                         # output size 7x7x64

## fc1 layer ##
W_fc1 = weight_variable([50*50*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 50*50*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## fc2 layer ##
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# the error between prediction and real data
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=ys, logits=prediction)
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction)))
# loss
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
# train_step = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

batch_xs=[]
batch_ys=[]
for fileName in os.listdir('trainImg'):
    batch_xs.append(mpimg.imread('trainImg/'+fileName))
    batch_ys.append(booleanMap[fileName.split('.')[0]])
    if(len(batch_ys)==100):
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        batch_xs=[]
        batch_ys=[]

In [7]:
test_xs=[]
test_ys=[]
for fileName in os.listdir('testImg'):
    test_xs.append(mpimg.imread('testImg/'+fileName))
    test_ys.append(booleanMap[fileName.split('.')[0]])
    y_pre = sess.run(prediction, feed_dict={xs: [mpimg.imread('testImg/'+fileName)], keep_prob: 1})
    print(y_pre[0][0])

y_pre = sess.run(prediction, feed_dict={xs: test_xs, keep_prob: 1})
correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(np.array(test_ys),1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={xs: test_xs, ys: test_ys, keep_prob: 1}))
sess.close()

1.0
1.0
1.0
1.0
0.0
1.0
5.187779e-29
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.99999785
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
3.3012673e-28
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
7.866453e-14
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1

MIT License

Copyright (c) 2018 Nik Bear Brown

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.